In [239]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

**1)** Давайте проанализируем данные опроса 4361 женщин из Ботсваны. О каждой из них мы знаем:
- Сколько детей она родила (признак ceb)
- Возраст (age)
- Длительность получения образования (educ)
- Религиозная принадлежность (religion)
- Идеальное, по её мнению, количество детей в семье (idlnchld)
- Была ли она когда-нибудь замужем (evermarr)
- Возраст первого замужества (agefm)
- Длительность получения образования мужем (heduc)
- Знает ли она о методах контрацепции (knowmeth)
- Использует ли она методы контрацепции (usemeth)
- Живёт ли она в городе (urban)
- Есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам. Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак ```religion```?

In [240]:
cd C:\Users\vlad\1. Machine Learning\Yandex Specialization\4. Statistics for Data Analysis\week 3\data

C:\Users\vlad\1. Machine Learning\Yandex Specialization\4. Statistics for Data Analysis\week 3\data


In [241]:
data = pd.read_csv('botswana.csv', sep='\t')
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [242]:
data['religion'].value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

**2)** Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [243]:
data.dropna()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
5,1,30,5,spirit,5.0,1.0,0.0,1,24.0,7.0,1,1.0,0.0,0.0,0.0
6,3,42,4,other,3.0,1.0,0.0,1,15.0,11.0,1,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353,9,49,0,protestant,5.0,0.0,0.0,1,15.0,0.0,0,0.0,1.0,0.0,0.0
4354,3,31,2,protestant,4.0,1.0,1.0,1,18.0,0.0,0,0.0,1.0,0.0,0.0
4355,4,27,6,protestant,4.0,1.0,1.0,1,17.0,7.0,0,0.0,0.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


In [244]:
# Отберем столбцы с пропущенными значениями
nan_columns = data.columns[data.isna().any()].tolist()
data[nan_columns].head()

,idlnchld,knowmeth,usemeth,agefm,heduc,electric,radio,tv,bicycle
0,4.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0
1,2.0,1.0,1.0,20.0,14.0,1.0,1.0,1.0,1.0
2,4.0,1.0,0.0,22.0,1.0,1.0,1.0,0.0,0.0
3,2.0,1.0,0.0,NaN,NaN,1.0,1.0,1.0,1.0
4,3.0,1.0,1.0,24.0,12.0,1.0,1.0,1.0,1.0


**3)** В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному. 

Например, в признаке ```agefm``` (возраст первого замужества) пропуски стоят только там, где ```evermarr=0``` (была ли она когда-нибудь замужем), то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака ```NaN``` соответствует значению **"не применимо".** 

В подобных случаях, когда признак ```x1``` на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

- Cоздать новый бинарный признак ```x2``` (1 - **"не применимо"**, 0 - иначе)
- Заменить **"не применимо"** в ```x1``` на произвольную константу ```c```, которая среди других значений ```x1``` не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида:
```y = β0 + β1*x1 + β2*x2```

на тех объектах, где ```x1``` было измерено, регрессионное уравнение примет вид: ```y = β0 + β1*x1```

а там, где x1 было **"не применимо"**, получится: ```y = β0 + β1*c + β2```

Выбор ```c``` влияет только на значение и интерпретацию ```β2```, но не ```β1``` 

Давайте используем этот метод для обработки пропусков в ```agefm``` и ```heduc```(длительность получения образования мужем):
- 1) Создайте признак ```nevermarr```, равный единице там, где в ```agefm``` пропуски.
- 2) Удалите признак ```evermarr``` — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
- 3) Замените NaN в признаке ```agefm``` c = 0
- 4) У объектов, где ```nevermarr = 1```, замените ```NaN``` в признаке ```heduc``` на ```c = -1```

Сколько осталось пропущенных значений в признаке ```heduc```?

In [245]:
# Создаем новый бинарный признак (1- "не применимо", 0 - иначе)
data['nevermarr'] = data['agefm'].apply(lambda x: 1 if np.isnan(x) else 0)

# Удаляем предыдущий признак, теперь он нам не нужен, иначе мультиколлинеарность
data.drop('evermarr', axis=1, inplace=True)

# Теперь можем заменить NaN для признака agefm (заменим на 0: уникальное для данного признака)
data['agefm'].fillna(0, inplace=True)

# Также можем заменить NaN для признака heduc (заменим на -1: уникальное для данного признака)
data.loc[data[data['nevermarr'] == 1]['heduc'].index, 'heduc'] = -1

**4)** Избавимся от оставшихся пропусков.

Для признаков ```idlnchld, heduc и usemeth``` проведите операцию, аналогичную предыдущей:

- Cоздайте индикаторы пропусков по этим признакам (```idlnchld_noans, heduc_noans, usemeth_noans```)
- Замените пропуски на нехарактерные значения: (c_idlnchld = -1, c_heduc = -2 (т.к. -1 уже использовали), с_usemeth = -1)

Остались только пропуски в признаках ```knowmeth, electric, radio, tv и bicycle```. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [205]:
# Индикаторы пропусков
data['idlnchld_no_answ'] = data['idlnchld'].apply(lambda x: 1 if np.isnan(x) else 0)
data['heduc_no_answ'] = data['heduc'].apply(lambda x: 1 if np.isnan(x) else 0)
data['usemeth_no_answ'] = data['usemeth'].apply(lambda x: 1 if np.isnan(x) else 0)

# Заменяем пропуски на нехарактерные значения
data['idlnchld'].fillna(-1, inplace=True)
data['heduc'].fillna(-2, inplace=True)
data['usemeth'].fillna(-1, inplace=True)

# В остальных признаках число пропусков незначительное не больше (7-10), удалим их
data.dropna(inplace=True)

data.shape

**5)** Постройте регрессию количества детей ```ceb``` на все имеющиеся признаки методом ```smf.ols```. Какой получился коэффициент детерминации R2. Округлите до трёх знаков после десятичной точки.

In [223]:
reg_model = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm +' \
                         'heduc + urban + electric + radio + tv + bicycle+' \
                         'nevermarr + idlnchld_no_answ + heduc_no_answ + usemeth_no_answ', data=data)

reg_model_fitted = reg_model.fit()
print(reg_model_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 25 Feb 2021   Prob (F-statistic):               0.00
Time:                        02:50:27   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

**6)** Обратите внимание, что для признака ```religion``` в модели **автоматически создалось несколько бинарных фиктивных переменных (Dummy Variables)** Сколько их?

**7)** Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1. 

Для проверки гомоскедастичности ошибок используем функцию Бройша-Пагана

H_0: остатки гомоскедастичны

In [217]:
print('Breusch-Pagan Criterion p-value: ', sms.het_breuschpagan(reg_model_fitted.resid, reg_model_fitted.model.exog)[1])

Breusch-Pagan Criterion p-value:  1.1452927633442407e-225


Остатки **не являются гомоскедастичными**, необходимо ввести поправку Уайта

In [224]:
reg_model_all = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm +' \
                         'heduc + urban + electric + radio + tv + bicycle+' \
                         'nevermarr + idlnchld_no_answ + heduc_no_answ + usemeth_no_answ', data=data)

reg_model_all_fitted = reg_model_all.fit(cov_type='HC1')
print(reg_model_all_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Thu, 25 Feb 2021   Prob (F-statistic):               0.00
Time:                        02:50:47   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

**8)** Удалите из модели незначимые признаки ```religion, radio и tv```. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта. 

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [225]:
# Обучаем модель без незначимых признаков 
reg_model_red = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm +' \
                         'heduc + urban + electric + bicycle+' \
                         'nevermarr + idlnchld_no_answ + heduc_no_answ + usemeth_no_answ', data=data)

reg_model_red_fitted = reg_model_red.fit()
print(reg_model_red_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Thu, 25 Feb 2021   Prob (F-statistic):               0.00
Time:                        02:51:31   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.0698      0.198  

In [226]:
print('Breusch-Pagan Criterion p-value: ', sms.het_breuschpagan(reg_model_red_fitted.resid, reg_model_red_fitted.model.exog)[1])

Breusch-Pagan Criterion p-value:  1.1197458896536614e-228


Ошибки по прежнему не являются гомоскедастичными

In [227]:
reg_model_red = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm +' \
                         'heduc + urban + electric + bicycle+' \
                         'nevermarr + idlnchld_no_answ + heduc_no_answ + usemeth_no_answ', data=data)

reg_model_red_fitted = reg_model_red.fit(cov_type='HC1')
print(reg_model_red_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 25 Feb 2021   Prob (F-statistic):               0.00
Time:                        02:52:53   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.0698      0.258  

Видим, что ухудшения модели не произошло. Проверим при помощи критерия Фишера

In [228]:
print('F=%f, p-value:%f, k1=%f' %reg_model_all.fit().compare_f_test(reg_model_red.fit()))

F=0.919236, p-value:0.467231, k1=5.000000


Достигаемый уровень значимости достаточно велик, следовательно, можем избавится от незначимых признаков

**9)** Признак ```usemeth_no_answ``` значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в ```usemeth``` произвольно выбранным значением ```c_usemeth = -1``` удалять ```usemeth_no_answ и usemeth``` можно только вместе.

Удалите из текущей модели ```usemeth_no_answ и usemeth```. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости.

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без ```usemeth_no_answ и usemeth```.  

In [231]:
# Удаляем usemeth_no_answ и usemeth
reg_model_red_2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm +' \
                         'heduc + urban + electric + bicycle+' \
                         'nevermarr + idlnchld_no_answ + heduc_no_answ', data=data)

reg_model_red_2_fitted = reg_model_red_2.fit(cov_type='HC1')
print(reg_model_red_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 25 Feb 2021   Prob (F-statistic):               0.00
Time:                        03:01:10   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.0698      0.258  

Проверим критерием Фишера значимость отличий 2-х моделей (ухудшилось ли качество)

In [238]:
print('F=%f, p-value:%f, k1=%f' %reg_model_red.fit().compare_f_test(reg_model_red_2.fit()))
print(reg_model_red.fit().compare_f_test(reg_model_red_2.fit()))

F=92.890582, p-value:0.000000, k1=2.000000
(92.89058230109664, 3.155200948041877e-40, 2.0)


Качество изменилось, вернем признаки обратно